> This kernel is based on the work of https://www.kaggle.com/thebrownviking20/bert-multiclass-classification

In [ ]:
import os
os.listdir('../input')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import sys
import zipfile
import datetime
print(os.listdir("../input"))
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))
train = pd.read_csv('../input/train_2kmZucJ.csv')
test = pd.read_csv('../input/test_oJQbWVk.csv')
df = train

ll = len(df['label'].unique())
df.head()

# Any results you write to the current directory are saved as output.
#     print(os.listdir(f"../input/{d}"))
# print(os.listdir("../input/googles-bert-model/chinese_l-12_h-768_a-12/chinese_L-12_H-768_A-12"))
# Any results you write to the current directory are saved as output.


# 1. Kernel Overview

## 1.1 Defination :

In today world** Text Classification/Segmentation/Categorization** (for example ticket categorization in a call centre, email classification, logs category detection etc.) is a common task. With humongous data out there, its nearly impossible to do this manually. Let's try to solve this problem automatically using machine learning and natural language processing tools.

## 1.2 Problem Statement

BBC articles dataset(2126 records) consist of two features text and the assiciated categories namely 
1. Sport 
2. Business 
3. Politics 
4. Tech 
5. Others

**Our task is to train a multiclass classification model on the mentioned dataset.**

## 1.3 Metrics

**Accuracy** - Classification accuracy is the number of correct predictions made as a
ratio of all predictions made

**Precision** - precision (also called positive predictive value) is the fraction of
relevant instances among the retrieved instances

**F1_score** - considers both the precision and the recall of the test to compute the
score

**Recall** – recall (also known as sensitivity) is the fraction of relevant instances that
have been retrieved over the total amount of relevant instances

**Why these metrics?** - We took Accuracy, Precision, F1 Score and Recall as metrics
for evaluating our model because accuracy would give an estimate of correct prediction. Precision would give us an estimate about the positive category predicted value i.e. how much our model is giving relevant result. F1 Score gives a clubbed estimate of precision and recall.Recall would provide us the relevant positive category prediction to the false negative and true positive category recognition results.

## 1.4 Machine Learning Model Considered:

We will be using **BERT Base uncased model** for this use case. 

Bert model working is not in the scope of this kernal. Kindly refer other external sources.

Download Bert Requirements

In [ ]:
# !wget https://github.com/charles9n/bert-sklearn.git

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

# 2. Data Exploration

### Step 2.1 Load Dataset

In [ ]:
import pandas as pd

data=df

In [ ]:
df.columns = ['category', 'text']

In [ ]:
data.head()

In [ ]:
data['category'].value_counts()

# 3. Implementation

### Step 2.2 Map Textual labels to numeric using Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
df2 = pd.DataFrame()
df2["text"] = data["text"]
df2["label"] = (data["category"])

In [ ]:
df2.head()

### Step 2.3 Divide dataset to test and train dataset

In [ ]:
test.columns = ['text']
test.head()

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df2["text"].values, df2["label"].values, test_size=0.0002, random_state=42) ;X_test = test['text'].values
X_train, X_test, y_train, y_test = train_test_split(df2["text"].values, df2["label"].values, test_size=0.2, random_state=42)

### Step 2.4 Setting up BERT Configurations

In [ ]:
import modeling
import optimization
import run_classifier
import tokenization

In [ ]:
import zipfile

folder = 'model_folder'
with zipfile.ZipFile("cased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [ ]:
BERT_MODEL = 'cased_L-12_H-768_A-12.zip'
BERT_PRETRAINED_DIR = f'{folder}/cased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

In [ ]:
# label_list

In [ ]:
import os
import tensorflow as tf

def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 4
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 8.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 150
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('cased')

label_list = [str(num) for num in range(ll) ]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

### Step 2.5 Train BERT model

In [ ]:
%%time
import datetime

print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))


In [ ]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""
    
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)
        all_label_ids.append(feature.label_id)

    def input_fn(params):
        """The actual input function."""
        print(params)
        batch_size = 500

        num_examples = len(features)

        d = tf.data.Dataset.from_tensor_slices({
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "segment_ids":
                tf.constant(
                    all_segment_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "label_ids":
                tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
        })

        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        return d
    return input_fn

### Step 2.6 Prediction on Test Dataset

In [ ]:

predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [ ]:
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# xg = LGBMClassifier()
# xg.fit(train_features, y_train)

In [ ]:
# import gc
# gc.enable()
# gc.collect()


In [ ]:
import numpy as np
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))
print(len(preds))


# 4. Results

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print("Accuracy of BERT is:",f1_score(y_test,preds, average='weighted'))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,preds))

>** Past Work mentioned on this dataset at max achieved 95.22 accuracies. BERT base model for the same, without any preprocessing and achieved 97.75 accuracies.**

In [ ]:
len(preds)

In [ ]:
sam = pd.read_csv('../input/sample_submission_LnhVWA4.csv')
sam.head()

In [ ]:
sam['label'] = preds

In [ ]:
sam.head()

In [ ]:
sam.to_csv('sub.csv', index=False)
from IPython.display import FileLinks
FileLinks('.')

# 5. Future Improvements on this kernel:

* Explore preprocessing steps on data.
* Explore other models as baseline.
* Make this notebook more informative and illustrative.
* Explaination on Bert Model.
* More time on data exploration
and many more...

# 6. References

https://www.kaggle.com/thebrownviking20/bert-multiclass-classification